In [1]:
import os
import json
import numpy as np
import pandas as pd
from utils.eval import *
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt


import glob
import librosa
import moviepy as mp
from tqdm import tqdm
from scipy.signal import correlate
from scipy.signal import find_peaks
from madmom.features.beats import RNNBeatProcessor, DBNBeatTrackingProcessor

In [2]:
a, b = 45, 140
tau = 0.13

accuracy_zero = evaluation_multi_segment("anchor_zero", "uni",a,b, tolerance= tau)
accuracy_peak= evaluation_multi_segment("anchor_peak", "uni",a,b, tolerance= tau)
accuracy_energy = evaluation_multi_segment("anchor_energy", "uni",a,b, tolerance= tau)



In [3]:
hits_zero   = accuracy_zero['bothhand_y_bothfoot_y_torso_y_uni']['hit_idx']
hits_peak   = accuracy_peak['bothhand_y_bothfoot_y_torso_y_uni']['hit_idx']
hits_energy = accuracy_energy['bothhand_y_bothfoot_y_torso_y_uni']['hit_idx']

hits_seg_zero = accuracy_zero['bothhand_y_bothfoot_y_torso_y_uni']['hit_seg_names']
hits_seg_peak = accuracy_peak['bothhand_y_bothfoot_y_torso_y_uni']['hit_seg_names']
hits_seg_energy = accuracy_energy['bothhand_y_bothfoot_y_torso_y_uni']['hit_seg_names']

hits_beat_pulse_zero = accuracy_zero['bothhand_y_bothfoot_y_torso_y_uni']['hit_beat_pulse']
hits_beat_pulse_peak = accuracy_peak['bothhand_y_bothfoot_y_torso_y_uni']['hit_beat_pulse']
hits_beat_pulse_energy = accuracy_energy['bothhand_y_bothfoot_y_torso_y_uni']['hit_beat_pulse']

hits_bpm_zero = accuracy_zero['bothhand_y_bothfoot_y_torso_y_uni']['hit_bpm']
hits_bpm_peak = accuracy_peak['bothhand_y_bothfoot_y_torso_y_uni']['hit_bpm']
hits_bpm_energy = accuracy_energy['bothhand_y_bothfoot_y_torso_y_uni']['hit_bpm']

est_bpm_zero = accuracy_zero['bothhand_y_bothfoot_y_torso_y_uni']['est_bpm']
est_bpm_peak = accuracy_peak['bothhand_y_bothfoot_y_torso_y_uni']['est_bpm']
est_bpm_energy = accuracy_energy['bothhand_y_bothfoot_y_torso_y_uni']['est_bpm']

ref_bpm_zero = accuracy_zero['bothhand_y_bothfoot_y_torso_y_uni']['ref_bpm']
ref_bpm_peak = accuracy_peak['bothhand_y_bothfoot_y_torso_y_uni']['ref_bpm']
ref_bpm_energy = accuracy_energy['bothhand_y_bothfoot_y_torso_y_uni']['ref_bpm']


In [15]:
est_bpm_zero[121]

111.0

In [22]:
idx = 124


print(est_bpm_zero[idx], hits_seg_zero[idx])
print(est_bpm_peak[idx], hits_seg_peak[idx])
print(est_bpm_energy[idx], hits_seg_energy[idx])


133.2 both_hand_y
135.0 both_foot_y
133.2 both_hand_y


In [ ]:
aist_2d = "./aist_dataset/video"
filename_zero   = accuracy_zero['bothhand_y_bothfoot_y_torso_y_uni']['filename']
filename_peak   = accuracy_peak['bothhand_y_bothfoot_y_torso_y_uni']['filename']
filename_energy = accuracy_energy['bothhand_y_bothfoot_y_torso_y_uni']['filename']

# Merge & remove duplicates
filename_list = np.concatenate((filename_zero,filename_peak,filename_energy), axis=0)
filename_list = list(set(filename_list))

audio_sr = 22050  # sampling rate for audio
motion_sr = 60    # mocap fps

cross_modal_results = []
completed_files = set()

for filename in tqdm(filename_list, total=len(filename_list)):   # ✅ no need enumerate
    
    # Replace cAll → c01
    vid_nm = filename.replace("cAll", "c01")
    
    # Match any channel: c01_dXX_ch*.mp4
    search_pattern = os.path.join(aist_2d, vid_nm[:-4] + "*.mp4")
    candidates = glob.glob(search_pattern)

    if filename in completed_files:
        continue   # ✅ cleaner

    if not candidates:
        raise FileNotFoundError(f"No matching video found for {vid_nm}")

    vid_path = candidates[0]
    video = mp.VideoFileClip(vid_path)

    # Build audio output path
    audio_path = f"./aist_dataset/audio/{filename[:-5]}.wav"

    # Avoid re-extracting audio if file already exists
    if not os.path.exists(audio_path):
        audio = video.audio
        audio.write_audiofile(audio_path, fps=audio_sr, logger=None)

    completed_files.add(filename)
    

## Circular Phase and Anchor Pulse-Beat Alignment

In [ ]:
aist_2d = "./aist_dataset/video"
hit_beat_pulse = accuracy_peak['bothhand_y_bothfoot_y_torso_y_uni']['hit_beat_pulse']
hit_anc_seq = accuracy_peak['bothhand_y_bothfoot_y_torso_y_uni']['hit_anc_seq']
filename_list = accuracy_peak['bothhand_y_bothfoot_y_torso_y_uni']['filename']

audio_sr = 22050  # sampling rate for audio
motion_sr = 60   # your mocap fps

cross_modal_results = []

for i, filename in tqdm(enumerate(filename_list), total=len(filename_list)):
    # vid_nm = filename.replace("cAll", "c01")
    # search_pattern = os.path.join(aist_2d, vid_nm[:-4] + "*.mp4")  # match any chXX
    # candidates = glob.glob(search_pattern)
    
    
    # if candidates:
    #     vid_path = candidates[0]  # take first match
    #     video = mp.VideoFileClip(vid_path)
    # else:
    #     raise FileNotFoundError(f"No matching video found for {vid_nm}")
    
    
    # audio = video.audio
    audio_path = f"./aist_dataset/audio/{filename[:-5]}.wav"
    # audio.write_audiofile(audio_path, fps=audio_sr, logger=None)
    
    #librrosa beat tracking
    # y, sr = librosa.load(audio_path, sr=audio_sr)
    # tempo_audio, beats_audio = librosa.beat.beat_track(y=y, sr=sr, units='time')
    # beat_pulse_audio = np.zeros(int(y.shape[0] / sr * motion_sr))
    # beat_frames_audio = (beats_audio * motion_sr).astype(int)
    # beat_pulse_audio[beat_frames_audio[beat_frames_audio < len(beat_pulse_audio)]] = 1
    
    #madmom beat tracking
    proc = DBNBeatTrackingProcessor(fps=100)
    act = RNNBeatProcessor()(audio_path)
    beats_audio = proc(act)                     # beat times in seconds
    tempo_audio = 60.0 / np.median(np.diff(beats_audio))  # BPM
    
    beat_pulse_audio = np.zeros(int(beats_audio[-1] * motion_sr) + 1)
    beat_frames_audio = (beats_audio * motion_sr).astype(int)
    beat_pulse_audio[beat_frames_audio] = 1
    
    
    beat_pulse_motion = hit_beat_pulse[i]
    # beat_pulse_motion = hit_anc_seq[i]     # anchor sequence
    
    min_len = min(len(beat_pulse_motion), len(beat_pulse_audio))
    beat_pulse_motion = beat_pulse_motion[:min_len]
    beat_pulse_audio = beat_pulse_audio[:min_len]
    
    
    corr = correlate(beat_pulse_motion - np.mean(beat_pulse_motion),
                     beat_pulse_audio - np.mean(beat_pulse_audio), mode='full')
    lag = np.argmax(corr) - (len(beat_pulse_motion) - 1)
    max_corr = np.max(corr) / (np.std(beat_pulse_motion) * np.std(beat_pulse_audio) * len(beat_pulse_motion))

    # Convert lag to seconds
    lag_sec = lag / motion_sr
    
    
    # Find motion beat times from gaussian-like pulse
    peaks_motion, _ = find_peaks(beat_pulse_motion, height=np.max(beat_pulse_motion)*0.5, distance=motion_sr*0.3)
    beats_motion = peaks_motion / motion_sr  # convert to seconds

    
    # --- 1. Circular correlation of beat phases ---

    circ_corr = np.nan
    if len(beats_audio) > 1 and len(beats_motion) > 1:
        audio_period = np.median(np.diff(beats_audio))
        motion_phase = (2 * np.pi * ((beats_motion % audio_period) / audio_period))
        audio_phase  = (2 * np.pi * ((beats_audio % audio_period) / audio_period))
        n = min(len(motion_phase), len(audio_phase))
        motion_phase, audio_phase = motion_phase[:n], audio_phase[:n]

        sin_x = np.sin(motion_phase - np.mean(motion_phase))
        sin_y = np.sin(audio_phase - np.mean(audio_phase))
        denom = np.sqrt(np.sum(sin_x ** 2) * np.sum(sin_y ** 2))
        if denom > 0:
            circ_corr = np.sum(sin_x * sin_y) / denom


    # --- 2. Coincident-beat fraction (±40 ms tolerance) ---

    tol = 0.12  # 40 ms
    count_coinc = 0
    for t_a in beats_audio:
        if np.any(np.abs(t_a - beats_motion) <= tol):
            count_coinc += 1
    beat_agreement = count_coinc / len(beats_audio) if len(beats_audio) > 0 else np.nan

    cross_modal_results.append({
        "filename": filename,
        "audio_tempo": tempo_audio,
        "corr_max": max_corr,
        "lag_sec": lag_sec,
        "circ_corr": circ_corr,
        "beat_agreement": beat_agreement
        
    })
    
# save dataframe
cross_modal_df = pd.DataFrame(cross_modal_results)
cross_modal_df.to_csv("./cross_modal_alignment_with_beat_pulse_peak.csv", index=False)


print(cross_modal_df[['corr_max','circ_corr','beat_agreement','lag_sec','audio_tempo']].describe().round(3))

# plt.hist(df['corr_max'], bins=20, edgecolor='black')
# plt.xlabel('Cross-modal correlation (ρ)')
# plt.ylabel('Count')
# plt.title('Distribution of Audio–Dance Rhythmic Correlation')
# plt.tight_layout()
# plt.show()

# accuracy_zero['bothhand_y_bothfoot_y_torso_y_uni'].keys()

In [ ]:
filename[:-5]

### Noise Injection

In [ ]:
import os
import glob
import librosa
import numpy as np
import moviepy as mp
from tqdm import tqdm
import soundfile as sf
from scipy.signal import correlate
from scipy.signal import find_peaks
from madmom.features.beats import RNNBeatProcessor, DBNBeatTrackingProcessor

aist_2d = "./aist_dataset/video"
hit_beat_pulse = accuracy_zero['bothhand_y_bothfoot_y_torso_y_uni']['hit_beat_pulse']
hit_anc_seq = accuracy_zero['bothhand_y_bothfoot_y_torso_y_uni']['hit_anc_seq']
filename_list = accuracy_zero['bothhand_y_bothfoot_y_torso_y_uni']['filename']

audio_sr = 22050  # sampling rate for audio
motion_sr = 60   # your mocap fps

cross_modal_results = []

for i, filename in tqdm(enumerate(filename_list), total=len(filename_list)):
    vid_nm = filename.replace("cAll", "c01")
    search_pattern = os.path.join(aist_2d, vid_nm[:-4] + "*.mp4")
    candidates = glob.glob(search_pattern)

    if not candidates:
        raise FileNotFoundError(f"No matching video found for {vid_nm}")

    vid_path = candidates[0]
    video = mp.VideoFileClip(vid_path)

    # Save clean audio
    audio = video.audio
    audio_path = f"./aist_dataset/audio/temp_audio_{i}.wav"
    audio.write_audiofile(audio_path, fps=audio_sr, logger=None)

    # load clean audio
    y, sr = librosa.load(audio_path, sr=audio_sr)
    snr_list = [20, 10, 5, 0]

    for snr in snr_list:

        # 1) Inject noise ----------------------------------------
        noise = np.random.randn(len(y))
        noise = noise / np.max(np.abs(noise))
        alpha = 10 ** (-snr / 20)
        y_noisy = y + alpha * noise
        y_noisy = y_noisy / np.max(np.abs(y_noisy))

        noisy_audio_path = f"./aist_dataset/audio_snr/temp_audio_{i}_snr{snr}.wav"
        sf.write(noisy_audio_path, y_noisy, sr)

        # 2) Beat tracking on NOISY audio ------------------------
        proc = DBNBeatTrackingProcessor(fps=100)
        act = RNNBeatProcessor()(noisy_audio_path)
        beats_audio_noisy = proc(act)

        # If beat tracker fails → skip safely
        if len(beats_audio_noisy) < 2:
            cross_modal_results.append({
                "filename": vid_nm,
                "snr": snr,
                "audio_tempo_noisy": np.nan,
                "corr_max": np.nan,
                "lag_sec": np.nan,
                "circ_corr": np.nan,
                "beat_agreement": np.nan
            })
            continue

        # 3) Compute noisy audio tempo
        tempo_audio = 60.0 / np.median(np.diff(beats_audio_noisy))

        # 4) Build beat pulse from noisy beats
        beat_pulse_audio = np.zeros(int(beats_audio_noisy[-1] * motion_sr) + 1)
        beat_frames_audio = (beats_audio_noisy * motion_sr).astype(int)
        beat_pulse_audio[beat_frames_audio] = 1

        # 5) Motion beat pulse
        beat_pulse_motion = hit_beat_pulse[i]

        # align lengths
        min_len = min(len(beat_pulse_motion), len(beat_pulse_audio))
        beat_pulse_motion = beat_pulse_motion[:min_len]
        beat_pulse_audio  = beat_pulse_audio[:min_len]

        # 6) Linear correlation
        corr = correlate(
            beat_pulse_motion - np.mean(beat_pulse_motion),
            beat_pulse_audio - np.mean(beat_pulse_audio),
            mode='full'
        )
        lag = np.argmax(corr) - (len(beat_pulse_motion) - 1)
        max_corr = np.max(corr) / (
            np.std(beat_pulse_motion) * np.std(beat_pulse_audio) * len(beat_pulse_motion)
        )
        lag_sec = lag / motion_sr

        # 7) Find motion beat times (using peaks)
        peaks_motion, _ = find_peaks(
            beat_pulse_motion,
            height=np.max(beat_pulse_motion) * 0.5,
            distance=motion_sr * 0.3
        )
        beats_motion = peaks_motion / motion_sr

        # 8) Circular correlation --------------------------------
        circ_corr = np.nan
        if len(beats_motion) > 1:
            audio_period = np.median(np.diff(beats_audio_noisy))
            motion_phase = 2 * np.pi * ((beats_motion % audio_period) / audio_period)
            audio_phase  = 2 * np.pi * ((beats_audio_noisy % audio_period) / audio_period)

            n = min(len(motion_phase), len(audio_phase))
            motion_phase, audio_phase = motion_phase[:n], audio_phase[:n]

            sin_x = np.sin(motion_phase - np.mean(motion_phase))
            sin_y = np.sin(audio_phase  - np.mean(audio_phase))
            denom = np.sqrt(np.sum(sin_x**2) * np.sum(sin_y**2))

            if denom > 0:
                circ_corr = np.sum(sin_x * sin_y) / denom

        # 9) Beat agreement (±100 ms) -----------------------------
        tol = 0.12
        beat_agreement = np.mean([
            np.any(np.abs(t_a - beats_motion) <= tol)
            for t_a in beats_audio_noisy
        ])

        # 10) Store results ---------------------------------------
        cross_modal_results.append({
            "filename": vid_nm,
            "snr": snr,
            "audio_tempo_noisy": tempo_audio,
            "corr_max": max_corr,
            "lag_sec": lag_sec,
            "circ_corr": circ_corr,
            "beat_agreement": beat_agreement
        })

    
# save dataframe
cross_modal_df = pd.DataFrame(cross_modal_results)
cross_modal_df.to_csv("./cross_modal_alignment_noise.csv", index=False)

df = pd.DataFrame(cross_modal_results)
print(df[['corr_max','circ_corr','beat_agreement','lag_sec','audio_tempo']].describe().round(3))

# plt.hist(df['corr_max'], bins=20, edgecolor='black')
# plt.xlabel('Cross-modal correlation (ρ)')
# plt.ylabel('Count')
# plt.title('Distribution of Audio–Dance Rhythmic Correlation')
# plt.tight_layout()
# plt.show()

# accuracy_zero['bothhand_y_bothfoot_y_torso_y_uni'].keys()

In [ ]:
df = pd.read_csv("./cross_modal_alignment_noise.csv")

In [ ]:
snr_summary = df.groupby("snr").agg({
    "audio_tempo_noisy": ["mean", "std"],
    "corr_max": ["mean", "std"],
    "lag_sec": ["mean", "std"],
    "circ_corr": ["mean", "std"],
    "beat_agreement": ["mean", "std"]
})

print(snr_summary)